In [1]:
import pandas as pd 

In [2]:
df = pd.read_excel('data.xlsx')

In [3]:
df.head(1)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0


Listado de cosas por preprocesar:
- ID, eliminar (identificador)
- Patient Sex (Transformación a Binario)
- Left-Fundus - Right-Fundus (matener)
- Left-Diagnostic Keywords - Right-Diagnostic Keywords (TOP-N Vectorizamos todas?)
- N	D G	C A	H M	O son el target
 

In [4]:
df = df.drop(columns=['ID'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Patient Age                3500 non-null   int64 
 1   Patient Sex                3500 non-null   object
 2   Left-Fundus                3500 non-null   object
 3   Right-Fundus               3500 non-null   object
 4   Left-Diagnostic Keywords   3500 non-null   object
 5   Right-Diagnostic Keywords  3500 non-null   object
 6   N                          3500 non-null   int64 
 7   D                          3500 non-null   int64 
 8   G                          3500 non-null   int64 
 9   C                          3500 non-null   int64 
 10  A                          3500 non-null   int64 
 11  H                          3500 non-null   int64 
 12  M                          3500 non-null   int64 
 13  O                          3500 non-null   int64 
dtypes: int64

In [6]:
#Transformar a binario el campo Patient Sex
df['Patient_Sex_Binario'] = df['Patient Sex'].map({'Male': 1, "Female": 0})


In [7]:
df.Patient_Sex_Binario.head(10)

0    0
1    1
2    1
3    1
4    1
5    0
6    1
7    0
8    1
9    1
Name: Patient_Sex_Binario, dtype: int64

In [8]:
df.drop(columns=['Patient Sex'], inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Patient Age                3500 non-null   int64 
 1   Left-Fundus                3500 non-null   object
 2   Right-Fundus               3500 non-null   object
 3   Left-Diagnostic Keywords   3500 non-null   object
 4   Right-Diagnostic Keywords  3500 non-null   object
 5   N                          3500 non-null   int64 
 6   D                          3500 non-null   int64 
 7   G                          3500 non-null   int64 
 8   C                          3500 non-null   int64 
 9   A                          3500 non-null   int64 
 10  H                          3500 non-null   int64 
 11  M                          3500 non-null   int64 
 12  O                          3500 non-null   int64 
 13  Patient_Sex_Binario        3500 non-null   int64 
dtypes: int64

- opcional crear columna categorica con la cantidad de enfermedades.
- opcional crear columna Binaria con si es lens dust o no 0/1

In [10]:

def parse_keywords(cell: str) -> list[str]:
    """
    Convierte la cadena 'laser spot, white vessel' --> ['laser spot', 'white vessel']
    Devuelve lista vacía si la celda está vacía o es NaN.
    """
    if pd.isna(cell):
        return []
    # divide por coma, quita espacios extra, pasa a minúsculas
    parts = [kw.strip().lower() for kw in cell.split(',')]
    # elimina strings vacíos (caso de ',,' → '' )
    return [p for p in parts if p]


In [11]:
df['kw_left']  = df['Left-Diagnostic Keywords'].apply(parse_keywords)
df['kw_right'] = df['Right-Diagnostic Keywords'].apply(parse_keywords)

# Combina listas: ojo, crea una NUEVA lista por fila (evita aliasing con .copy())
df['kw_all'] = df.apply(lambda row: row['kw_left'] + row['kw_right'], axis=1)


In [12]:
from itertools import chain

all_keywords = sorted(set(chain.from_iterable(df['kw_all'])))
# 0 lo reservamos para padding; empezamos en 1
stoi = {kw: idx + 1 for idx, kw in enumerate(all_keywords)}
itos = {i: w for w, i in stoi.items()}
PAD_IDX = 0


In [13]:
df['kw_idx'] = df['kw_all'].apply(lambda kws: [stoi[k] for k in kws])


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Patient Age                3500 non-null   int64 
 1   Left-Fundus                3500 non-null   object
 2   Right-Fundus               3500 non-null   object
 3   Left-Diagnostic Keywords   3500 non-null   object
 4   Right-Diagnostic Keywords  3500 non-null   object
 5   N                          3500 non-null   int64 
 6   D                          3500 non-null   int64 
 7   G                          3500 non-null   int64 
 8   C                          3500 non-null   int64 
 9   A                          3500 non-null   int64 
 10  H                          3500 non-null   int64 
 11  M                          3500 non-null   int64 
 12  O                          3500 non-null   int64 
 13  Patient_Sex_Binario        3500 non-null   int64 
 14  kw_left 

In [19]:
df.head(5)           


,Patient Age,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,Patient_Sex_Binario,kw_left,kw_right,kw_all,kw_idx
0,69,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,0,[cataract],[normal fundus],"[cataract, normal fundus]","[9, 235]"
1,57,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,1,[normal fundus],[normal fundus],"[normal fundus, normal fundus]","[235, 235]"
2,42,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,1,[laser spot，moderate non proliferative retinop...,[moderate non proliferative retinopathy],[laser spot，moderate non proliferative retinop...,"[117, 184]"
3,66,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1,1,[normal fundus],[branch retinal artery occlusion],"[normal fundus, branch retinal artery occlusion]","[235, 6]"
4,53,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,1,[macular epiretinal membrane],[mild nonproliferative retinopathy],"[macular epiretinal membrane, mild nonprolifer...","[151, 171]"


Revisar Embedding de Indices si individual o por conjuntos-debatir**